# Import librerie e definizione funzioni d'utilità

In [ ]:
import functions as fn

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import geopandas as gpd

In [ ]:
dataset_name = "ebusiness"
if not os.path.exists("..\\Plot"):
    os.mkdir("..\\Plot")
if not os.path.exists("..\\Plot\\" + dataset_name):
    os.mkdir("..\\Plot\\" + dataset_name)
if not os.path.exists("..\\Plot\\" + dataset_name + "\\tempi"):
    os.mkdir("..\\Plot\\" + dataset_name + "\\tempi")
if not os.path.exists("..\\DatasetParsed"):
    os.mkdir("..\\DatasetParsed")
if not os.path.exists("..\\DatasetParsed\\" + dataset_name):
    os.mkdir("..\\DatasetParsed\\" + dataset_name)

In [ ]:
counter = 0
def make_path(dataset, file_name, other = ""):
    global counter
    counter += 1
    return "..\\Plot\\" + dataset + "\\" + other + str(counter) + "-" + file_name

# Estrazione dati tabella aziende

In [ ]:
dataset_eb = "..\\Datasets\\" + dataset_name + "\\ebusiness-aziende-dataset.csv"
df_eb = pd.read_csv(dataset_eb, encoding='cp1252')
df_time = df_eb[["URL", "Tempo richiesta", "Tempo estrazione Name", "Tempo estrazione Code", "Tempo estrazione Legal Form", "Tempo estrazione Status", "Tempo estrazione Reg. Date", "Tempo estrazione Capital", "Tempo estrazione Address", "Tempo estrazione Del. Date", "Tempo estrazione dati Rapp.", "Tempo estrazione Partners", "Tempo estrazione Activity"]]
df_eb = df_eb[["URL", "ID", "Name", "Code", "Legal form", "Status", "Registration Date", "Capital", "Address", "Deletion Date"]]
print("Numero entry tabella originale: ", len(df_eb))

In [ ]:
def eek_to_eur(x):
    # 1 EEK --> 0,0639115 EUR
    return x * 0.0639115

def convert_capital(x):
    if type(x) == str:
        x = x.replace("Capitalis", "")
        x = x.replace("€", "")
        x = x.replace("\xa0", "")
        if "EEK" in x:
            x = x.replace("EEK", "")
            return eek_to_eur(float(x))
    return float(x)

In [ ]:
df_eb['Capital'] = df_eb['Capital'].apply(convert_capital)

In [ ]:
df_eb.to_csv("..\\DatasetParsed\\" + dataset_name + "\\" + dataset_name + "-aziende.csv", index=False)
df_eb.head()

In [ ]:
info_dataset = fn.extract_data_from_df(df_eb)
display(pd.DataFrame(info_dataset, index=["valore"]).T)

### Statistiche celle

In [ ]:
keys = ["celle totali", "celle nulle"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle nulle"]
]
title = "Confronto numero celle totali e vuote"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["celle totali", "celle str", "celle float", "celle int"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle str"],
    info_dataset["numero celle float"],
    info_dataset["numero celle int"]
]
title = "Confronto tipo celle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche righe

In [ ]:
keys = ["righe totali", "righe con celle nulle"]
values = [
    info_dataset["numero righe"], 
    info_dataset["numero righe con celle nulle"]
]
title = "Confronto numero righe totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche colonne

In [ ]:
keys = ["colonne totali", "colonne con celle nulle"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne con celle nulle"]
]
title = "Confronto numero colonne totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["colonne totali", "colonne str", "colonne float", "colonne int"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne str"],
    info_dataset["numero colonne float"],
    info_dataset["numero colonne int"]
]
title = "Confronto tipo colonne"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
different_value_columns = {}
for key in df_eb:
    different_value_columns[key] = [len(pd.unique(df_eb[key])), df_eb[key].isnull().sum()]
different_value_columns = dict(sorted(different_value_columns.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys = different_value_columns.keys()
values = [item[0] for item in different_value_columns.values()]
title = "Numero di valori distinti per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

In [ ]:
keys = different_value_columns.keys()
values = [item[1] for item in different_value_columns.values()]
title = "Numero di valori nulli per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

### Plot su mappe geografiche

In [ ]:
region = [x.split("maakond")[0].strip() + " maakond" for x in list(df_eb["Address"]) if x is not np.nan]
n_companies_by_region = {}
res = []
[res.append(x) for x in region if x not in res]
for c in res:
    n_companies_by_region[c] = region.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\Estonia\\maakond_20221101.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[0] in n_companies_by_region:
        map_df.at[i, 'n_companies'] = n_companies_by_region[row[0]]
map_df["MNIMI"] = map_df["MNIMI"].apply(lambda x: x.split(" ")[0])
map_df

In [ ]:
fn.map_plot(map_df, "n_companies", "MNIMI", "geometry", 10,"Numero di aziende per contea", make_path(dataset_name, "contee_estonia"))

# Estrazione dati tabella activity

In [ ]:
dataset_eb_a = "..\\Datasets\\" + dataset_name + "\\ebusiness-activity-dataset.csv"
dataset_eb_p = "..\\Datasets\\" + dataset_name + "\\ebusiness-partners-dataset.csv"
dataset_eb_r = "..\\Datasets\\" + dataset_name + "\\ebusiness-rapp-dataset.csv"

In [ ]:
df_eb_a = pd.read_csv(dataset_eb, encoding='cp1252')
df_time = df_eb[["URL", "Tempo richiesta", "Tempo estrazione Name", "Tempo estrazione Code", "Tempo estrazione Legal Form", "Tempo estrazione Status", "Tempo estrazione Reg. Date", "Tempo estrazione Capital", "Tempo estrazione Address", "Tempo estrazione Del. Date"]]
df_eb_a = df_eb[["URL", "ID", "Name", "Code", "Legal form", "Status", "Registration Date", "Capital", "Address", "Deletion Date"]]
print("Numero entry tabella originale: ", len(df_eb))

# Estrazione dati tabella partners

# Estrazione dati tabella rappresentanti giuridici

# Estrazione prestazioni

In [ ]:
# Tempo totale di esecuzione 1590207327300: (nanosecondi), 1468 entry

In [ ]:
df_time.head()

In [ ]:
df_time.describe()

In [ ]:
new_df_time = df_time.describe()
del new_df_time["Tempo richiesta"]

In [ ]:
keys = ["Tempo richiesta", "Tempo estrazione dettagli"]
values = [df_time.describe()["Tempo richiesta"]["mean"], new_df_time.sum(axis="columns")["mean"] / 1000000000]
values = np.array(values) * 1000
title = "Confronto tempo medio (ms) request vs tempo medio (ms) estrazione dettagli"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["mean"].keys())
values = list(new_df_time.loc["mean"])
values = np.array(values) / 1000000
title = "Tempi medi (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["max"].keys())
values = list(new_df_time.loc["max"])
values = np.array(values) / 1000000
title = "Tempo massimo (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)